In [13]:
import pandas as pd
from collections import Counter
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import hdbscan
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
import torch.optim as optim
import wandb 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the Datasets

In [3]:
train = pd.read_pickle("train.pk")
val = pd.read_pickle("val.pk")
test = pd.read_pickle("test.pk")

In [4]:
len(train["BookingCode"].unique())

100

In [5]:
full_ds = pd.concat([train, val, test])

label_dict = {i:  booking_code for i, booking_code in enumerate(full_ds["BookingCode"].unique().tolist())}
booking_code_to_label = {v: k for k, v in label_dict.items()}

# PartNumber
part_number_dict = {i:  part_number for i, part_number in enumerate(full_ds["PartNumber"].unique().tolist())}
part_number_to_label = {v: k for k, v in part_number_dict.items()}

# hdbscanCluster
cluster_dict = {i:  cluster_number for i, cluster_number in enumerate(full_ds["hdbscanCluster"].unique().tolist())}
cluster_to_label = {v: k for k, v in cluster_dict.items()}

In [6]:
# create the dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, preprocess_transforms=None):
        self.raw_data = dataframe
        self.preprocess_transforms = preprocess_transforms
        self.part_num_classes = len(part_number_dict)
        self.cluster_classes = len(cluster_dict)

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        row = self.raw_data.iloc[idx]

        text_emb = np.array(row["descriptionEmbeddings"])
        part_num_idx = part_number_to_label[row["PartNumber"]]
        price_norm = row["NormPrice"]
        cluster_idx = cluster_to_label[row["hdbscanCluster"]]

        # One-hot encode part_num and cluster
        part_num_one_hot = np.zeros(self.part_num_classes)
        part_num_one_hot[part_num_idx] = 1

        cluster_one_hot = np.zeros(self.cluster_classes)
        cluster_one_hot[cluster_idx] = 1

        vector = np.concatenate([text_emb, part_num_one_hot, [price_norm], cluster_one_hot])

        label = booking_code_to_label[row["BookingCode"]]

        sample = {
            "vector": vector,
            "label": label
        }

        # Apply preprocessing transforms if provided
        if self.preprocess_transforms is not None:
            sample = self.preprocess_transforms(sample)

        return sample



In [7]:
train_ds = CustomDataset(train)
val_ds = CustomDataset(val)
test_ds = CustomDataset(test)

In [9]:
def dataset_to_numpy(dataset):
    """
    Convert a PyTorch Dataset into NumPy arrays for vectors and labels.
    
    Args:
        dataset (Dataset): The dataset to convert.
    
    Returns:
        tuple: A tuple containing:
            - vectors: A 2D NumPy array where each row is a feature vector.
            - labels: A list of corresponding labels.
    """
    vectors = []
    labels = []

    for sample in dataset:
        vectors.append(sample["vector"])
        labels.append(sample["label"])

    # Stack vectors into a single NumPy array
    vectors = np.stack(vectors)

    return vectors, labels


In [12]:
train_ds_arr, train_labels = dataset_to_numpy(train_ds)
val_ds_arr, val_labels = dataset_to_numpy(val_ds)
test_ds_arr, test_labels = dataset_to_numpy(test_ds)

In [10]:
train.columns

Index(['ID', 'PartNumber', 'Description', 'Count', 'SumPrice', 'BookingCode',
       'DocumentId', 'descriptionEmbeddings', 'Price', 'NormPrice',
       'hdbscanCluster'],
      dtype='object')

# Training the SVM One-VS-REST

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)
